# Visualize Landprices


In [1]:
import pandas as pd
import geopandas as gpd
import os

C:\Users\BerkÖztürk\AppData\Local\Temp\ipykernel_7964\1734800707.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


### Vector data 
We start by reading and quickly visualising vector data

### Read raster data

We read the census data that is classified in a grid, then we read the grid as a geopandas file.

In [2]:
ZENSUS_PATH = r"../data/raw/2 Zensus/"
NEIGHBORHOOD_PATH =  r"../data/raw/3 Neighborhoods"
LAND_PRICES_PATH = r"../data/raw/1 Land Prices"
CITY_NAMES=["Berlin","Bremen","Dresden","Frankfurt","Köln"]
zensus_files= os.listdir(ZENSUS_PATH)
neighborhood_files= os.listdir(NEIGHBORHOOD_PATH)
landprices_files= os.listdir(LAND_PRICES_PATH)

def get_file_names(): 
    city_files={}
    for city_name in CITY_NAMES:
        
        csv_fpaths= [os.path.join(ZENSUS_PATH,fpath) for fpath in zensus_files if ( city_name in fpath ) and (fpath.endswith(".csv")) ]
        gpkg_fpaths= [os.path.join(ZENSUS_PATH,fpath) for fpath in zensus_files if ( city_name in fpath ) and (fpath.endswith(".gpkg")) ]
        
        neighbourhood_fpaths= [os.path.join(NEIGHBORHOOD_PATH,fpath) for fpath in neighborhood_files if ( city_name in fpath ) and (fpath.endswith(".gpkg")) ]
        landprices_fpaths= [os.path.join(LAND_PRICES_PATH,fpath) for fpath in landprices_files if ( city_name in fpath ) ]

        city_files[city_name]= (csv_fpaths,gpkg_fpaths,neighbourhood_fpaths,landprices_fpaths)
    return city_files

def combine_data_within_city(city_files):
    city_merged_data={}
    for city_name, (csv_fpaths,gpkg_fpaths,neighbourhood_fpaths,landprices_fpaths) in city_files.items():
        df_list_city=[]
        for csv_fpath in csv_fpaths:
            df = pd.read_csv(csv_fpath, sep=";", encoding="utf-8-sig").drop(columns="Unnamed: 0")
            df_list_city.append(df)

        from functools import reduce
        zensus_csv_merged = reduce(lambda df1,df2: pd.merge(df1,df2,on="Grid_Code"), df_list_city)
        
        # read grid
        grid_city = gpd.read_file(gpkg_fpaths[0])
        grid_city = grid_city.merge(zensus_csv_merged, on = "Grid_Code")

        # merge files
        prices_city = pd.read_csv(landprices_fpaths[0], sep=";", encoding= "utf-8-sig").drop(columns="Unnamed: 0")
        neighborhood_city = gpd.read_file(neighbourhood_fpaths[0])
        neighborhood_city = neighborhood_city.merge(prices_city, on = "Neighborhood_FID", how = "inner")

        neighborhood_city_4326 = neighborhood_city.to_crs(epsg = 4326)
        amenity_features = get_amenity_features(neighborhood_city_4326)

        # Perform spatial join using sjoin
        merged_data = gpd.sjoin(grid_city, neighborhood_city, how='left', op='intersects')
        merged_data = merged_data.drop(columns=["City_Name_y", "City_Code_right"]).rename(columns={"City_Name_x":"City_Name", "City_Code_left":"City_Code"})
        city_merged_data[city_name] = merged_data
    return city_merged_data

def concat_city_dataframes(city_merged_data):
    col_list=[]
    for city_name,df in city_merged_data.items():
        print(df.shape)
        col_list.append(df.columns.to_list())
    common_cols=list(set(col_list[0]).intersection(*col_list))

    grid_level_df_list = [df[common_cols] for df in city_merged_data.values()]
    grid_level_all_cities = pd.concat(grid_level_df_list, axis=0)
    return grid_level_all_cities

def get_amenity_features(neighborhoods):
    # Define the tag to extract (amenity=restaurant)
    tag = {'amenity':True}

    # Define an empty list to store the results
    results = {}

    # Loop over each neighborhood and extract the restaurants
    for i, nb_name in enumerate(neighborhoods.Neighborhood_Name):
        nb = neighborhoods.loc[neighborhoods.Neighborhood_Name == nb_name]
        restaurants = ox.geometries.geometries_from_polygon(polygon=nb.geometry.iloc[0], tags=tag)
        # print(f'Processed {i+1}/{len(neighborhoods)} neighborhoods ({nb_name}): found {restaurants.shape[0]} restaurants')
        results[nb_name] = restaurants

    nb_results = []

    for i, nb_name in enumerate(neighborhoods.Neighborhood_Name):
        nb_result = pd.DataFrame(results[nb_name].amenity.value_counts().to_dict(), index=[i])
        nb_result["Neighborhood_Name"]=nb_name
        nb_result["City_Name"]=neighborhoods.City_Name
        nb_results.append(nb_result)

    # Combine the results into a single GeoDataFrame
    combined_results = pd.concat(nb_results, ignore_index=True)
    

    DROPNA_TRESH=0.8
    combined_results = combined_results.loc[:, df.isnull().mean() < DROPNA_TRESH]

    return combined_results 

In [ ]:
city_files= get_file_names()


city_merged_data = combine_data_within_city(city_files)

grid_level_all_cities= concat_city_dataframes(city_merged_data)


In [ ]:
splitted_area_names=pd.Series(grid_level_all_cities.Area_Types.unique()).str.split("_")
unique_area_cols = pd.Series(np.concatenate(splitted_area_names)).unique().tolist()
for unique_area_col in unique_area_cols:
    grid_level_all_cities['is_{}'.format(unique_area_col)]= grid_level_all_cities.Area_Types.str.contains(unique_area_col).astype(int)
grid_level_all_cities = grid_level_all_cities.drop(columns='Area_Types')

In [51]:
import numpy as np


numeric_cols = grid_level_all_cities.select_dtypes(include=['int32','int64']).columns
non_numeric_cols = grid_level_all_cities.select_dtypes(exclude=['int32','int64']).columns

agg_operations= dict(zip(numeric_cols, ['mean']*len(numeric_cols) ))
agg_operations["Land_Value"]="first"
grid_level_all_cities_neighborhood_level =  grid_level_all_cities.groupby(['City_Name','Neighborhood_Name']).agg(agg_operations).reset_index()
#grid_level_all_cities_neighborhood_level= grid_level_all_cities_neighborhood_level.drop(columns='index_right')#.describe()

In [53]:
grid_level_all_cities_neighborhood_level.to_csv('../data/interim/nb_level_merged_all_cities.csv',index=False)

In [ ]:
# Save merged data to file
grid_level_all_cities.to_file('../data/interim/grid_level_merged_all_cities.gpkg', driver='GPKG')

In [ ]:
        neighborhood_city = gpd.read_file(neighbourhood_fpaths[0])
        neighborhood_city = neighborhood_city.merge(prices_city, on = "Neighborhood_FID", how = "inner")

        neighborhood_city_4326 = neighborhood_city.to_crs(epsg = 4326)
        amenity_features = get_amenity_features(neighborhood_city_4326)
